<a href="https://colab.research.google.com/github/CoelhoNunes/Deep_Learning_Spring_2024/blob/main/Assignment_7_CNN_for_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1: Importing dependencies

In [205]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

from google.colab import drive

In [206]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Stage 2: Data preprocessing

## Loading files

In [207]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [208]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
train_data = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/trainingandtestdata/training.1600000.processed.noemoticon.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [209]:
train_data.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Preprocessing

In [211]:
data = train_data

### Cleaning

In [212]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [213]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, "lxml").get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)
  return tweet

In [214]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [215]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [216]:
set(data_labels)

{0, 1}

### Tokenization

In [217]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

### Padding

In [218]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

### Spliting into training/testing set

In [219]:
test_idx = np.random.randint(0, 800000, 8000)
text_idx = np.concatenate((test_idx, test_idx + 800000))

In [220]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

# Stage 3: Model building

In [254]:
class DCNN(tf.keras.Model):

  def __init__(self,
               vocab_size,
               emb_dim=128,
               nb_filters=50,
               FFN_units=512,
               nb_classes=2,
               dropout_rate=0.1,
               training=False,
               name="dcnn"):
    super(DCNN, self).__init__(name=name)

    self.embedding = layers.Embedding(vocab_size + 1, # Add 1 to vocab_size to accommodate padding token
                                      emb_dim)
    self.bigram = layers.Conv1D(filters=nb_filters,
                                kernel_size=2,
                                padding="same", # Changed padding to "same" to avoid dimension reduction
                                activation="relu")
    self.pool_1 = layers.GlobalMaxPool1D()
    self.trigram = layers.Conv1D(filters=nb_filters,
                                 kernel_size=3,
                                 padding="same", # Changed padding to "same" to avoid dimension reduction
                                 activation="relu")
    self.pool_2 = layers.GlobalMaxPool1D()
    self.fourgram = layers.Conv1D(filters=nb_filters,
                                 kernel_size=4,
                                 padding="same", # Changed padding to "same" to avoid dimension reduction
                                 activation="relu")
    self.pool_3 = layers.GlobalMaxPool1D()
    self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
    self.dropout = layers.Dropout(rate=dropout_rate)
    if nb_classes == 2:
      self.last_dense = layers.Dense(units=1,
                                     activation="sigmoid")
    else:
      self.last_dense = layers.Dense(units=nb_classes,
                                     activation="softmax")

  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool_1(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool_2(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool_3(x_3)

    # Check if any of the feature vectors are empty
    if x_1.shape[1] == 0 or x_2.shape[1] == 0 or x_3.shape[1] == 0:
        # If any are empty, return a tensor of zeros with the expected shape
        return tf.zeros((inputs.shape[0], self.last_dense.units), dtype=tf.float32)

    merged = tf.concat([x_1, x_2, x_3], axis=-1)
    merged = self.dense_1(merged)
    merged = self.dropout(merged, training=training) # Pass 'training' as a keyword argument
    output = self.last_dense(merged)

    return output

# class DCNN(tf.keras.Model):

#   def __init__(self,
#                vocab_size,
#                emb_dim=128,
#                nb_filters=50,
#                FFN_units=512,
#                nb_classes=2,
#                dropout_rate=0.1,
#                training=False,
#                name="dcnn"):
#     super(DCNN, self).__init__(name=name)

#     self.embedding = layers.Embedding(vocab_size,
#                                       emb_dim)
#     self.bigram = layers.Conv1D(filters=nb_filters,
#                                 kernel_size=2,
#                                 padding="valid",
#                                 activation="relu")
#     self.pool_1 = layers.GlobalMaxPool1D()
#     self.trigram = layers.Conv1D(filters=nb_filters,
#                                  kernel_size=3,
#                                  padding="valid",
#                                  activation="relu")
#     self.pool_2 = layers.GlobalMaxPool1D()
#     self.fourgram = layers.Conv1D(filters=nb_filters,
#                                  kernel_size=4,
#                                  padding="valid",
#                                  activation="relu")
#     self.pool_3 = layers.GlobalMaxPool1D()
#     self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
#     self.dropout = layers.Dropout(rate=dropout_rate)
#     if nb_classes == 2:
#       self.last_dense = layers.Dense(units=1,
#                                      activation="sigmoid")
#     else:
#       self.last_dense = layers.Dense(units=nb_classes,
#                                      activation="softmax")

#   def call(self, inputs, training):
#     x = self.embedding(inputs)
#     x_1 = self.bigram(x)
#     x_1 = self.pool_1(x_1)
#     x_2 = self.trigram(x)
#     x_2 = self.pool_2(x_2)
#     x_3 = self.fourgram(x)
#     x_3 = self.pool_3(x_3)

#     merged = tf.concat([x_1, x_2, x_3], axis=-1)
#     merged = self.dense_1(merged)
#     merged = self.dropout(merged, training=training) # Pass 'training' as a keyword argument
#     output = self.last_dense(merged)

#     return output

# Stage 4: Application

## Config

In [255]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = len(set(train_labels))
DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHES = 5

## Training

In [256]:


Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [257]:
if NB_CLASSES == 2:
  Dcnn.compile(loss="binary_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])

else:
  Dcnn.compile(loss="sparse_categorical_crossentropy",
               optimizer="adam",
               metrics=["sparse_categorical_accuracy"])

In [258]:
checkpoint_path = "/content/drive/MyDrive/CNN_for_NLP/ckpt"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Latest checkpoint restored!")

Latest checkpoint restored!


In [ ]:
Dcnn.fit(train_inputs,
          train_labels,
          batch_size=BATCH_SIZE,
          epochs=NB_EPOCHES)
ckpt_manager.save()

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['dcnn/embedding_19/embeddings', 'dcnn/dense_39/kernel', 'dcnn/dense_39/kernel', 'dcnn/dense_39/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


49752/49752 ━━━━━━━━━━━━━━━━━━━━ 13173s 265ms/step - accuracy: 0.8020 - loss: 0.4257
Epoch 2/5
49752/49752 ━━━━━━━━━━━━━━━━━━━━ 12056s 241ms/step - accuracy: 0.8592 - loss: 0.3299
Epoch 3/5
14148/49752 ━━━━━━━━━━━━━━━━━━━━ 2:23:41 242ms/step - accuracy: 0.8892 - loss: 0.2697

## Evaluation

In [ ]:
Dcnn(np.array([tokenizer.encode("You are so nice")]), training=False).numpy()